In [2]:
import requests
import getpass
import socket
import json
import zipfile
import io
import math
import shutil
import pprint
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
import h5py
import re
from glob import glob
import numpy as np

import os
import gdal

import rasterio as rio
from rasterio.plot import show

%matplotlib widget
#import earthpy.Spatial as es

# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
from statistics import mean
from requests.auth import HTTPBasicAuth

In [3]:
import datetime as dt

# File name format: ATL06_[yyyymmdd][hhmmss]_[RGTccss]_[vvv_rr].h5

#NOTE: Need to simplify this function
def time_from_fname(fname):
    """ IS2 fname -> datatime object. """
    t = fname.split('_')[1]
    y, m , d, h, mn, s = t[:4], t[4:6], t[6:8], t[8:10], t[10:12], t[12:14]
    time = dt.datetime(int(y), int(m), int(d), int(h), int(mn), int(s))
    return time


def segment_from_fname(fname):
    """ IS2 fname -> segment number. """
    s = fname.split('_')[2]
    return int(s[-2:])


def select_files(files, segments=[10,11,12], t1=(2019,1,1), t2=(2019,2,1)):
    t1 = dt.datetime(*t1)
    t2 = dt.datetime(*t2)
    files_out = []
    for f in files:
        fname = os.path.basename(f)
        time = time_from_fname(fname)
        segment = segment_from_fname(fname)
        if t1 <= time <= t2 and segment in segments:
            files_out.append(f)
    return files_out

In [4]:
import pyproj
from astropy.time import Time

def gps2dyr(time):
    """ Converte GPS time to decimal years. """
    return Time(time, format='gps').decimalyear


def track_type(time, lat, tmax=1):
    """
    Separate tracks into ascending and descending.
    
    Defines tracks as segments with time breaks > tmax,
    and tests whether lat increases or decreases w/time.
    """
    tracks = np.zeros(lat.shape)  # generate track segment
    tracks[0:np.argmax(np.abs(lat))] = 1  # set values for segment
    i_asc = np.zeros(tracks.shape, dtype=bool)  # output index array

    # Loop trough individual secments
    for track in np.unique(tracks):
    
        i_track, = np.where(track == tracks)  # get all pts from seg
    
        if len(i_track) < 2: continue
    
        # Test if lat increases (asc) or decreases (des) w/time
        i_min = time[i_track].argmin()
        i_max = time[i_track].argmax()
        lat_diff = lat[i_track][i_max] - lat[i_track][i_min]
    
        # Determine track type
        if lat_diff > 0:  i_asc[i_track] = True
    
    return i_asc, np.invert(i_asc)  # index vectors


def transform_coord(proj1, proj2, x, y):
    """
    Transform coordinates from proj1 to proj2 (EPSG num).

    Example EPSG projs:
        Geodetic (lon/lat): 4326
        Polar Stereo AnIS (x/y): 3031
        Polar Stereo GrIS (x/y): 3413
    """
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+str(proj1))
    proj2 = pyproj.Proj("+init=EPSG:"+str(proj2))
    return pyproj.transform(proj1, proj2, x, y)  # convert


In [5]:
import h5py
import numpy as np

def read_atl06(fname, bbox=None):
    """ 
    Read 1 ATL06 file and output 6 reduced files. 
    
    Extract variables of interest and separate the ATL06 file 
    into each beam (ground track) and ascending/descending orbits.
    """

    # Each beam is a group
    group = ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']

    # Loop trough beams
    for k,g in enumerate(group):
    
        #-----------------------------------#
        # 1) Read in data for a single beam #
        #-----------------------------------#
    
        # Load variables into memory (more can be added!)
        with h5py.File(fname, 'r') as fi:
            lat = fi[g+'/land_ice_segments/latitude'][:]
            lon = fi[g+'/land_ice_segments/longitude'][:]
            h_li = fi[g+'/land_ice_segments/h_li'][:]
            s_li = fi[g+'/land_ice_segments/h_li_sigma'][:]
            t_dt = fi[g+'/land_ice_segments/delta_time'][:]
            q_flag = fi[g+'/land_ice_segments/atl06_quality_summary'][:]
            s_fg = fi[g+'/land_ice_segments/fit_statistics/signal_selection_source'][:]
            snr = fi[g+'/land_ice_segments/fit_statistics/snr_significance'][:]
            h_rb = fi[g+'/land_ice_segments/fit_statistics/h_robust_sprd'][:]
            dac = fi[g+'/land_ice_segments/geophysical/dac'][:]
            f_sn = fi[g+'/land_ice_segments/geophysical/bsnow_conf'][:]
            dh_fit_dx = fi[g+'/land_ice_segments/fit_statistics/dh_fit_dx'][:]
            tide_earth = fi[g+'/land_ice_segments/geophysical/tide_earth'][:]
            tide_load = fi[g+'/land_ice_segments/geophysical/tide_load'][:]
            tide_ocean = fi[g+'/land_ice_segments/geophysical/tide_ocean'][:]
            tide_pole = fi[g+'/land_ice_segments/geophysical/tide_pole'][:]
            t_ref = fi['/ancillary_data/atlas_sdp_gps_epoch'][:]
            rgt = fi['/orbit_info/rgt'][:] * np.ones(len(lat))
            orb = np.full_like(h_li, k)

        #---------------------------------------------#
        # 2) Filter data according region and quality #
        #---------------------------------------------#
        
        # Select a region of interest
        if bbox:
            lonmin, lonmax, latmin, latmax = bbox
            bbox_mask = (lon >= lonmin) & (lon <= lonmax) & \
                        (lat >= latmin) & (lat <= latmax)
        else:
            bbox_mask = np.ones_like(lat, dtype=bool)  # get all
            
        # Only keep good data, and data inside bbox
        mask = (q_flag == 0) & (np.abs(h_li) < 10e3) & (bbox_mask == 1)
        
        # Update variables
        lat, lon, h_li, s_li, t_dt, h_rb, s_fg, snr, q_flag, f_sn, \
            tide_earth, tide_load, tide_ocean, tide_pole, dac, rgt, orb = \
                lat[mask], lon[mask], h_li[mask], s_li[mask], t_dt[mask], \
                h_rb[mask], s_fg[mask], snr[mask], q_flag[mask], f_sn[mask], \
                tide_earth[mask], tide_load[mask], tide_ocean[mask], \
                tide_pole[mask], dac[mask], rgt[mask], orb[mask]

        # Test for no data
        if len(h_li) == 0: continue

        #-------------------------------------#
        # 3) Convert time and separate tracks #
        #-------------------------------------#
        
        # Time in GPS seconds (secs sinde 1980...)
        t_gps = t_ref + t_dt

        # Time in decimal years
        t_year = gps2dyr(t_gps)

        # Determine orbit type
        i_asc, i_des = track_type(t_year, lat)
        
        #-----------------------#
        # 4) Save selected data #
        #-----------------------#
        
        # Define output file name
        ofile = fname.replace('.h5', '_'+g[1:]+'.h5')
                
        # Save variables
        with h5py.File(ofile, 'w') as f:
            f['orbit'] = orb
            f['lon'] = lon
            f['lat'] = lat
            f['h_elv'] = h_li
            f['t_year'] = t_year
            f['t_sec'] = t_gps
            f['s_elv'] = s_li
            f['h_rb'] = h_rb
            f['s_fg'] = s_fg
            f['snr'] = snr
            f['q_flg'] = q_flag
            f['f_sn'] = f_sn
            f['tide_load'] = tide_load
            f['tide_ocean'] = tide_ocean
            f['tide_pole'] = tide_pole
            f['tide_earth'] = tide_earth
            f['dac'] = dac
            f['rgt'] = rgt
            f['trk_type'] = i_asc

            print('out ->', ofile)
                

In [28]:
import h5py
import numpy as np

def read_atl03(fname, bbox=None):
    print(fname)
    """ 
    Read 1 ATL06 file and output 6 reduced files. 
    
    Extract variables of interest and separate the ATL06 file 
    into each beam (ground track) and ascending/descending orbits.
    """
    f = h5py.File(fname,'r')
    print(f.keys())
    # Each beam is a group
    group = ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
    # Loop trough beams
    for k,g in enumerate(group):
    
        #-----------------------------------#
        # 1) Read in data for a single beam #
        #-----------------------------------#
    
        # Load variables into memory (more can be added!)
        with h5py.File(fname, 'r') as fi:
            if g+'/heights' in fi.keys():
                lat = fi[g+'/heights/lat_ph'][:]
                lon = fi[g+'/heights/lon_ph'][:]
                h = fi[g+'/heights/h_ph'][:]
                conf = fi[g+'/heights/signal_conf_ph']
                bkg = fi[g+'/bckgrd_atlas/bckgrd_counts'][:]

                land_ice_class = conf[:,3]
          
        #-----------------------------------#
        # 3) Filter data #
        #-----------------------------------#
                mask = (land_ice_class == 4) & (np.abs(h) < 10e3)
                lat,lon,h = lat[mask],lon[mask],h[mask]
        
        #-----------------------#
        # 4) Save selected data #
        #-----------------------#
        
        # Define output file name
                ofile = fname.replace('.h5', '_'+g[1:]+'.h5')
                
        # Save variables
                with h5py.File(ofile, 'w') as f:
                    f['lon'] = lon
                    f['lat'] = lat
                    f['h_elv'] = h
                    f['bkc_ct'] = bkg
            
                    print('out ->', ofile)
                

In [15]:
import getpass


def list_files_local(path):
    """ Get file list form local folder. """
    from glob import glob
    return glob(path)


In [16]:

# !pwd
# files = list_files_local('../data/RBIS/*ATL03*01.h5')


# del files[files.index('../data/RBIS/processed_ATL03_20181202114602_09900112_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20181230213426_00370210_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20190111210924_02200210_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20181231102210_00450212_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20181214112102_11730112_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20190128201036_04790210_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20190112095707_02280212_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20181213223319_11650110_001_01.h5')]
# del files[files.index('../data/RBIS/processed_ATL03_20190214082459_07310212_001_01.h5')]

# njobs = 1#len(files)

# #NOTE: Using Kamb bounding box for now
# bbox = None #[-1124782, 81623, -919821, -96334]

# if njobs == 1:
#     print('running in serial ...')
#     [read_atl03(f, bbox) for f in files]

# else:
#     print('running in parallel (%d jobs) ...' % njobs)
#     from joblib import Parallel, delayed
#     Parallel(n_jobs=njobs, verbose=5)(delayed(read_atl06)(f, bbox) for f in files)


In [29]:

#!pwd
files = list_files_local('../data/RLIS/*ATL03*01.h5')

#print(files)

njobs = 1#len(files)

#NOTE: Using Kamb bounding box for now
bbox = None #[-1124782, 81623, -919821, -96334]

if njobs == 1:
    print('running in serial ...')
    [read_atl03(f, bbox) for f in files]

else:
    print('running in parallel (%d jobs) ...' % njobs)
    from joblib import Parallel, delayed
    Parallel(n_jobs=njobs, verbose=5)(delayed(read_atl06)(f, bbox) for f in files)


running in serial ...
../data/RLIS/processed_ATL03_20181224114725_13260112_001_01.h5
<KeysViewHDF5 ['METADATA', 'ancillary_data', 'atlas_impulse_response', 'ds_surf_type', 'ds_xyz', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']>
[2439 2655 2840 ...  776  846  815]
out -> ../data/RLIS/processed_ATL03_20181224114725_13260112_001_01_gt1l.h5
[2781 2815 2827 ... 1388 1429 1347]
out -> ../data/RLIS/processed_ATL03_20181224114725_13260112_001_01_gt1r.h5
[2912 2841 2792 ...  705  741  643]
out -> ../data/RLIS/processed_ATL03_20181224114725_13260112_001_01_gt2l.h5
[2594 2570 2614 ...  997 1007  958]
out -> ../data/RLIS/processed_ATL03_20181224114725_13260112_001_01_gt2r.h5
[2655 2706 2678 ...  933  896  940]
out -> ../data/RLIS/processed_ATL03_20181224114725_13260112_001_01_gt3l.h5
[3286 3213 3151 ... 1290 1197 1273]
out -> ../data/RLIS/processed_ATL03_20181224114725_13260112_001_01_gt3r.h5
../data/RLIS/processed_ATL03_20190212091617_07010212_001_01.h5
<Key

In [38]:
import matplotlib.pyplot as plt
%matplotlib widget

def read_h5(fname, vnames=[]):
    """ Simple HDF5 reader. """
    with h5py.File(fname, 'r') as f:
        return [f[v][:] for v in vnames]

    
files = list_files_local('../data/RLIS/*ATL03*gt*')

#33
#2
#6
#18
#23
#54


lon, lat, h,bkg = read_h5(files[54], ['lon', 'lat', 'h_elv','bkc_ct'])
print(lon.shape,h.shape,bkg.shape)

plt.plot(bkg)
# x_ice, y_ice = transform_coord(4326, 3031, lon, lat)
 
# plt.figure()
# plt.plot(h,'.',markersize = 0.1)

(131554,) (131554,) (85044,)


FigureCanvasNbAgg()

In [14]:
image1_bands = glob("*.TIF")
image1_bands.sort()

image1_blue = np.squeeze(rio.open(image1_bands[0]).read())
image1_red = np.squeeze(rio.open(image1_bands[2]).read())
image1_green = np.squeeze(rio.open(image1_bands[1]).read())

image2_blue = np.squeeze(rio.open(image1_bands[3]).read())
image2_red = np.squeeze(rio.open(image1_bands[5]).read())
image2_green = np.squeeze(rio.open(image1_bands[4]).read())

#image1_nir = np.squeeze(rio.open(image1_bands[3]).read())
    
#Normalize bands into 0.0 - 1.0 scale
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize band DN
blue = normalize(image1_blue)
red = normalize(image1_red)
green = normalize(image1_green)
#nir = normalize(image1_nir)

#ndwi = (green - nir)/(green - nir)
# Stack bands
rgb = np.dstack((red, green, blue))
#print(nrg.shape())
# View the color composite
rgb = rgb.astype(float)
#plt.imshow(rgb)
#es.plot_bands(image1_blue[0],title="Landsat Cropped Band 4\nColdsprings Fire Scar",cmap="Greys_r")

In [75]:
print(rgb.shape)

(9161, 9181, 3)


In [16]:
# Load raster and metadata
tile_path = 'LC08_L1GT_167109_20190125_20190205_01_T2_B2.TIF'
Raster = gdal.Open(tile_path)
width = Raster.RasterXSize
height = Raster.RasterYSize
gt = Raster.GetGeoTransform()
array = Raster.ReadAsArray()

# Pixel numbers
x = np.arange(0, width)
y = np.arange(0, height)

# Grid Cell Coordinates of upper left corner in EPSG:3031 UTM. 
X = gt[0] + x * gt[1] 
Y = gt[3] + y * gt[5]

#xx,yy = np.meshgrid(X,Y)

#lon, lat = transform_coord(3031, 4326, xx, yy)



In [1]:
# %matplotlib inline

# # Load "Natural Earth” countries dataset, bundled with GeoPandas
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# # Overlay glacier outline
# f, ax = plt.subplots(1, figsize=(12, 6))
# world.plot(ax=ax, facecolor='lightgray', edgecolor='gray')
plt.figure()
plt.imshow(rgb, extent = [np.min(X), np.max(X), np.min(Y), np.max(Y)])
plt.plot(x_ice, y_ice,'r')
# ax.set_ylim([-71.5, -67.5])
# ax.set_xlim([7.5,15.5]);

NameError: name 'plt' is not defined